In [10]:
import os
import pandas as pd

base_dir = os.getcwd()
xlsm_path = os.path.join(base_dir, 'Amtliche Fassung des ATC-Index 2025.xlsm')
xls = pd.ExcelFile(xlsm_path, engine='openpyxl')
print("Sheet-Namen in der Datei:", xls.sheet_names)




Sheet-Namen in der Datei: ['Urheberrrechte', 'amtl. Index 2025 ATC-sortiert', 'amtl. Index 2025 alphabetisch ', 'Erläuterungen pflanzliche ATC', 'Erläuterungen']


In [12]:
import os
import pandas as pd

# Absoluter Pfad zum Arbeitsverzeichnis
base_dir = '/Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs'
xlsm_path = os.path.join(base_dir, 'Amtliche Fassung des ATC-Index 2025.xlsm')

# Sheet-Name exakt so wie in der Datei (inkl. Leerzeichen)
sheet_name = 'amtl. Index 2025 alphabetisch '
df = pd.read_excel(xlsm_path, sheet_name=sheet_name, engine='openpyxl')

# Nur die Spalte 'ATC-Bedeutung' auswählen
col = 'ATC-Bedeutung'
if col in df.columns:
    df_col = df[[col]]
    csv_path = os.path.join(base_dir, 'ATC-Bedeutung.csv')
    df_col.to_csv(csv_path, sep=';', index=False, encoding='utf-8')
    print(f"CSV mit Spalte '{col}' erstellt: {csv_path}")
    df_col.head()
else:
    print(f"Spalte '{col}' nicht gefunden. Verfügbare Spalten sind:\n{df.columns.tolist()}")



CSV mit Spalte 'ATC-Bedeutung' erstellt: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/ATC-Bedeutung.csv


In [13]:
import pandas as pd

# 1. CSV laden
csv_path = 'ATC-Bedeutung.csv'  # passe den Pfad ggf. an
df = pd.read_csv(csv_path, sep=';', encoding='utf-8')

# 2. Zeilen zählen vor dem Bereinigen
print(f"Anzahl Zeilen vor Entfernen der Duplikate: {len(df)}")

# 3. Duplikate entfernen (alle Spalten)
df_clean = df.drop_duplicates()

# 4. Zeilen zählen nach dem Bereinigen
print(f"Anzahl Zeilen nach Entfernen der Duplikate: {len(df_clean)}")

# 5. Optional: in eine neue CSV schreiben
df_clean.to_csv('ATC-Bedeutung_dedup.csv', sep=';', index=False, encoding='utf-8')


Anzahl Zeilen vor Entfernen der Duplikate: 7436
Anzahl Zeilen nach Entfernen der Duplikate: 6027


In [14]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur deduplizierten CSV
csv_path = os.path.join(base_dir, 'ATC-Bedeutung_dedup.csv')

# 3. CSV laden
df = pd.read_csv(csv_path, sep=';', encoding='utf-8')

# 2. Funktion zur Textbereinigung
def clean_text(text):
    if pd.isna(text):
        return text
    # in Kleinbuchstaben
    text = text.lower()
    # Umlaute ersetzen
    umlaut_map = {
        'ä': 'ae', 'ö': 'oe', 'ü': 'ue',
        'Ä': 'Ae', 'Ö': 'Oe', 'Ü': 'Ue', 'ß': 'ss'
    }
    for uml, rep in umlaut_map.items():
        text = text.replace(uml, rep)
    # Bindestriche und Klammern durch Leerzeichen
    text = text.replace('-', ' ')
    text = text.replace('(', ' ').replace(')', ' ')
    # Stop-Wörter entfernen (als eigene Worte)
    stopw = ['oder', 'und', 'in']
    for w in stopw:
        text = re.sub(r'\b' + w + r'\b', ' ', text)
    # Alle Sonderzeichen, Interpunktion und Ziffern entfernen
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Mehrfache Leerzeichen auf Einfach-Leerzeichen reduzieren
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 3. Neue Spalte anlegen
df['ATC-Bedeutung_cleaned'] = df['ATC-Bedeutung'].apply(clean_text)

# 4. Ergebnis prüfen
print(df[['ATC-Bedeutung', 'ATC-Bedeutung_cleaned']].head(10))

# 5. Optional: speichern
df.to_csv('ATC-Bedeutung_cleaned.csv', sep=';', index=False, encoding='utf-8')

    ATC-Bedeutung ATC-Bedeutung_cleaned
0             NaN                   NaN
1        Abacavir              abacavir
2    Abaloparatid          abaloparatid
3      Abametapir            abametapir
4        Abarelix              abarelix
5       Abatacept             abatacept
6       Abciximab             abciximab
7     Abemaciclib           abemaciclib
8        Abetimus              abetimus
9  Abicipar pegol        abicipar pegol


In [15]:
import os
import pandas as pd

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur bisherigen semikolon-getrennten CSV
csv_in = os.path.join(base_dir, 'ATC-Bedeutung_cleaned.csv')

# 3. Einlesen (Semikolon als Trenner)
df = pd.read_csv(csv_in, sep=';', encoding='utf-8')

# 4. Count vor dem Entfernen leerer Zeilen
print(f"Zeilen vor Bereinigung: {len(df)}")

# 5. Alle Zeilen entfernen, die in allen Spalten NaN sind
df_clean = df.dropna(how='all')

# 6. Count nachher
print(f"Zeilen nach Entfernen leerer Zeilen: {len(df_clean)}")

# 7. Abspeichern mit Komma-Delimiter (Standard)
csv_out = os.path.join(base_dir, 'ATC-Bedeutung_final.csv')
df_clean.to_csv(csv_out, index=False, encoding='utf-8')

print(f"Bereinigte CSV gespeichert: {csv_out}")


Zeilen vor Bereinigung: 6027
Zeilen nach Entfernen leerer Zeilen: 6026
Bereinigte CSV gespeichert: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/ATC-Bedeutung_final.csv


In [18]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Eingangs-CSV und Ausgangspfade
csv_in  = os.path.join(base_dir, 'deDE15LinguisticVariant.csv')
csv_out = os.path.join(base_dir, 'deDE15LinguisticVariant_final.csv')

# 3. Nur die Spalte COMPONENT einlesen
df = pd.read_csv(csv_in, usecols=['COMPONENT'], encoding='utf-8')

# 4. Funktion zur Textbereinigung für COMPONENT_cleaned
def clean_text(text):
    if pd.isna(text):
        return text
    # 1) Kleinbuchstaben
    text = text.lower()
    # 2) Umlaute ersetzen
    umlaut_map = {'ä':'ae','ö':'oe','ü':'ue','ß':'ss'}
    for uml, rep in umlaut_map.items():
        text = text.replace(uml, rep)
    # 3) Stop-Wörter entfernen
    for w in ['oder','und','in']:
        text = re.sub(r'\b{}\b'.format(w), ' ', text)
    # 4) "XXX" entfernen
    text = text.replace('xxx', ' ')
    # 5) Bindestriche und Klammern durch Leerzeichen
    text = text.replace('-', ' ')
    text = text.replace('(', ' ').replace(')', ' ')
    # 6) Sonderzeichen, Interpunktion, Zahlen und ^ durch Leerzeichen ersetzen
    text = re.sub(r'[^a-z\s]', ' ', text)
    # 7) Mehrfache Leerzeichen reduzieren und trimmen
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 5. Neue Spalte anlegen
df['COMPONENT_cleaned'] = df['COMPONENT'].apply(clean_text)

# 6. Speichern des neuen CSV
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Neues CSV erstellt: {csv_out} ({len(df)} Zeilen)")



Neues CSV erstellt: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/deDE15LinguisticVariant_final.csv (18257 Zeilen)


In [20]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Eingangs- und Ausgangs-CSV
csv_in  = os.path.join(base_dir, 'deDE15LinguisticVariant.csv')
csv_out = os.path.join(base_dir, 'deDE15LinguisticVariant_final.csv')

# 3. Nur die Spalte COMPONENT einlesen
df = pd.read_csv(csv_in, usecols=['COMPONENT'], encoding='utf-8')

# 4. Funktion zur Textbereinigung für COMPONENT_cleaned
def clean_text(text):
    if pd.isna(text):
        return text
    # Kleinbuchstaben
    text = text.lower()
    # Umlaute ersetzen
    umlaut_map = {'ä':'ae','ö':'oe','ü':'ue','ß':'ss'}
    for uml, rep in umlaut_map.items():
        text = text.replace(uml, rep)
    # Stop-Wörter entfernen
    for w in ['oder','und','in']:
        text = re.sub(r'\b{}\b'.format(w), ' ', text)
    # "XXX" entfernen
    text = text.replace('xxx', ' ')
    # Bindestriche und Klammern durch Leerzeichen
    text = text.replace('-', ' ')
    text = text.replace('(', ' ').replace(')', ' ')
    # Sonderzeichen, Interpunktion, Zahlen und ^ durch Leerzeichen ersetzen
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Einzelne Buchstaben entfernen (Wörter mit genau einem Zeichen)
    text = re.sub(r'\b[a-z]\b', ' ', text)
    # Mehrfache Leerzeichen reduzieren und trimmen
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 5. Neue Spalte anlegen
df['COMPONENT_cleaned'] = df['COMPONENT'].apply(clean_text)

# 6. Speichern des neuen CSV
df.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Neues CSV erstellt: {csv_out} ({len(df)} Zeilen)")


Neues CSV erstellt: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/deDE15LinguisticVariant_final.csv (18257 Zeilen)


In [21]:
import os
import pandas as pd

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfade definieren
csv_in  = os.path.join(base_dir, 'deDE15LinguisticVariant_final.csv')
csv_out = os.path.join(base_dir, 'deDE15LinguisticVariant_final_nodup.csv')

# 3. CSV laden
df = pd.read_csv(csv_in, usecols=['COMPONENT', 'COMPONENT_cleaned'], encoding='utf-8')

# 4. Count vor dem Entfernen der Duplikate
print(f"Zeilen insgesamt vor Dedup: {len(df)}")

# 5. Duplikate anhand der bereinigten Spalte entfernen
df_nodup = df.drop_duplicates(subset=['COMPONENT_cleaned'])

# 6. Count nachher
print(f"Zeilen insgesamt nach Dedup: {len(df_nodup)}")

# 7. Ergebnis speichern
df_nodup.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Dedupliziertes CSV gespeichert: {csv_out}")


Zeilen insgesamt vor Dedup: 18257
Zeilen insgesamt nach Dedup: 6073
Dedupliziertes CSV gespeichert: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/deDE15LinguisticVariant_final_nodup.csv


In [35]:
import os
import pandas as pd
import re

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur finalen CSV
csv_path = os.path.join(base_dir, 'ATC-Bedeutung_final.csv')

# 3. CSV einlesen (Standard-Komma-Delimiter)
df = pd.read_csv(csv_path, encoding='utf-8')

# 4. Stopwörter definieren (einschließlich 'andere' und 'ohne')
stopw = ['mit', 'von', 'und', 'bei', 'andere', 'ohne', 'anderen', 'exkl', 'nicht', 'haltige', 'zubereitung', 'kombinationen', 'haltige', 'verbindungen']

def further_clean(text):
    if pd.isna(text):
        return text
    # alles in Kleinbuchstaben
    text = text.lower()
    # 1) Stopwörter entfernen
    pattern_stop = r'\b(?:' + '|'.join(stopw) + r')\b'
    text = re.sub(pattern_stop, ' ', text)
    # 2) Einzelne Buchstaben entfernen
    text = re.sub(r'\b[a-z]\b', ' ', text)
    # 3) Mehrfache Leerzeichen auf ein Leerzeichen reduzieren und trimmen
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 5. Spalte updaten
df['ATC-Bedeutung_cleaned'] = df['ATC-Bedeutung_cleaned'].apply(further_clean)

# 6. Änderung speichern
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"Spalte 'ATC-Bedeutung_cleaned' wurde aktualisiert und '{csv_path}' gespeichert.")







Spalte 'ATC-Bedeutung_cleaned' wurde aktualisiert und '/Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/ATC-Bedeutung_final.csv' gespeichert.


In [36]:
import os
import pandas as pd

# 1. Arbeitsverzeichnis ermitteln
base_dir = os.getcwd()

# 2. Pfad zur aktuellen CSV
csv_in  = os.path.join(base_dir, 'ATC-Bedeutung_final.csv')

# 3. CSV einlesen
df = pd.read_csv(csv_in, encoding='utf-8')

# 4. Count vor Dedup
print(f"Zeilen vor Entfernen der Duplikate: {len(df)}")

# 5. Duplikate auf Basis der bereinigten Spalte entfernen
df_nodup = df.drop_duplicates(subset=['ATC-Bedeutung_cleaned'])

# 6. Count nachher
print(f"Zeilen nach Entfernen der Duplikate: {len(df_nodup)}")

# 7. Ausgabe-Dateiname mit Prefix
csv_out = os.path.join(base_dir, 'lexikon_' + os.path.basename(csv_in))

# 8. Speichern
df_nodup.to_csv(csv_out, index=False, encoding='utf-8')
print(f"Bereinigtes Lexikon gespeichert als: {csv_out}")


Zeilen vor Entfernen der Duplikate: 6026
Zeilen nach Entfernen der Duplikate: 5477
Bereinigtes Lexikon gespeichert als: /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/make_korpus/Procedures_Drugs/lexikon_ATC-Bedeutung_final.csv
